In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np 

In [ ]:
from main import main
main()

# Uncurated codes

In [ ]:
kitti360_root = "dataset/KITTI-360"
sequence = "2013_05_28_drive_0000_sync"
sp = sequence_processor = SequenceProcessor(kitti360_root, sequence)




In [ ]:
sp = sequence_processor
sp.get_persp_img(traj_i=5000, if_crop=True, show=True)
sp.original_persp_img_size, sp.image.shape

In [ ]:
from xgutils.vis import visutil
import numpy as np
from labels import semantics2rgbimg, kitti360_sequences
import matplotlib.pyplot as plt
from xgutils.vis import visutil
from xgutils import sysutil
import scipy
import os
from scipy import ndimage
def add_traj(img, xx, xt, resolution=(256,256)):
    fig, ax = visutil.newPlot(resolution=resolution)
    ax.imshow(img, extent=[-1,1,-1,1])
    ax.scatter(xx[:,0], xx[:,1], zorder=100, color='r', s=3.5)
    cmap = plt.get_cmap('viridis')
    for i in range(len(xx)):
        ax.plot(   [xx[i,0],  xt[i,0]],
                    [xx[i,1],  xt[i,1]], color=cmap(i/len(xx)))
    ax.set_xlim(-1,1)
    ax.set_ylim(-1,1)
    ax.axis('off')
    img = visutil.fig2img(fig)
    return img
def render_imgs(dset_dir="output",
                    sequence="2013_05_28_drive_0000_sync", 
                    trajlist = range(400), version=0):
    imgs = []
    for traj_i in sysutil.progbar(trajlist):
        fpath = "%s/labels/%s_%08d/boxes.npz"%(dset_dir, sequence, traj_i)
        save_path = "%s/datavis/imgs/%s/%08d.png"%(dset_dir, sequence, traj_i)
        if os.path.exists(save_path):
            continue
        img = plt.imread("%s/images/%s_%08d/0000.png"%(dset_dir, sequence, traj_i))
        img = img[:,:,:-1]

        loaded = np.load(fpath)
        img2 = loaded["layout"][0]
        img2 = semantics2rgbimg(img2, vis_color=True)
        ccds, tcds = loaded["camera_coords"], loaded["target_coords"]
        if version == 0:
            ccds, tcds = ccds[:,[0,2,1]], tcds[:,[0,2,1]]
        img2 = add_traj(img2, ccds, tcds, resolution = (img2.shape[0], img2.shape[1]))

        img2 = img2[:,:,:-1]
        img3 = loaded["layout_noveg"][0]
        img3 = semantics2rgbimg(img3, vis_color=True)/256.

        imgGrid = visutil.imageGrid([img, img2, img3], shape=(1, -1))
        imgs.append(imgGrid)
    return imgs
sequence = "2013_05_28_drive_0000_sync"
ftpt="output/kitti360_v1/filters/%s.npz"
lbpt="output/kitti360_v%d/labels/2013_05_28_drive_0000_sync_%08d/boxes.npz"
impt="output/kitti360_v%d/images/2013_05_28_drive_0000_sync_%08d/0000.png"
flist = range(100)
imgs0 = render_imgs(dset_dir="output/kitti360_v0/", sequence=sequence, trajlist=flist)
imgs1 = render_imgs(dset_dir="output/kitti360_v1/", sequence="2013_05_28_drive_0000_sync", trajlist=flist, version=1)
std_filter = np.load(ftpt%sequence)["std_filter"]
imgs1f= render_imgs(dset_dir="output/kitti360_v1/", sequence="2013_05_28_drive_0000_sync", trajlist=np.where(std_filter)[0][:len(flist)], version=1)

imgs = [ visutil.imageGrid([imgs0[i], imgs1[i], imgs1f[i]], shape=(-1,1)) for i in range(len(imgs0))]
# fid = 0
# imgs = []
# for fid in range(0, 1000, 1):
#     loaded0 = np.load(lbpt%(0, fid))
#     loaded1 = np.load(lbpt%(0, fid))
#     im0 = plt.imread(impt%(0, fid))
#     im1 = plt.imread(impt%(1, fid))
#     img = visutil.imageGrid([im0, im1], shape=(1,-1))
#     imgs.append(img)
visutil.imgarray2video( "temp/test.mp4", imgs, frameRate=60)

In [ ]:
#plt.scatter(xx[:,0], xx[:,2], zorder=100, color='r', s=0.5)
xx,xt=sequence_processor.get_next_cameras(traj_i=130,max_dist=125, cam_num=35)
xt = xx + xt
pm = sequence_processor.poses_matrices[130:150,:3,3]
#plt.scatter(pm[:,0], pm[:,1], zorder=100, color='r', s=0.5)
xx,xt=sequence_processor.export_next_cams(traj_i=120,)
plt.scatter(xx[:,0], xx[:,2], zorder=100, color='r', s=0.5)


In [ ]:
np.linalg.norm(xx[1:] - xx[:-1], axis=1).sum()

In [ ]:
from kitti360scripts.helpers.project import CameraPerspective
from kitti360_processor import normalize_intrinsics, denormalize_intrinsics
np.set_printoptions(precision=3, suppress=True)
for ixx in range(1):
    #traj_i = 17
    traj_i = 17 + ixx*100
    ocamp = CameraPerspective( kitti360_root, sequence, cam_id=0)
    camp = CameraPerspective( kitti360_root, sequence, cam_id=0)
    newK = camp.K.copy()
    print(camp.width, camp.height)
    #for ixx in range(15):
    #    newdx, newdy = 1408-ixx*97, 376
    newdx, newdy = 376, 376
    newdim = 256
    newdx, newdy = newdim, newdim
    newK = normalize_intrinsics(ocamp.K, (camp.width, camp.height))

    newK = np.array(    [   [0.39243911, 0.        , 0.48441012, 0.        ],
                            [0.        , 0.39243911, 0.6350254 , 0.        ],
                            [0.        , 0.        , 1.        , 0.        ],
                            [0.        , 0.        , 0.        , 1.        ],]

    newK = denormalize_intrinsics(newK, (newdx, newdy))
    # newK[:2,2] = ocamp.K[:2,2] / np.array([ocamp.width / newdx, ocamp.height / newdy])
    # newK[0,0] = ocamp.K[0,0] / ocamp.height * newdy
    # newK[1,1] = ocamp.K[1,1] / ocamp.height * newdy
    # print("K", ocamp.K)
    # print("newK",newK)

    #newK[0,2] = 
    #newK = camp.K*np.array([[1/camp.width, 1/camp.height, 1]]).T * 256
    # newK = camp.K
    # newK = normalize_intrinsics(newK, (camp.width, camp.height))
    # newK = denormalize_intrinsics(newK, (256, 256))
    camp.K = newK
    camp.width, camp.height = newdx, newdy
    sequence_processor.get_persp_img(traj_i=traj_i, if_crop=False, show=False);
    sequence_processor.perspect_plot_matplotlib(traj_i=traj_i, max_dist=50., camera = ocamp)
    #sequence_processor.get_persp_img(traj_i=traj_i, if_crop=True, crop_shape=(newdx, newdy), resize_to=(newdy, newdx, ), show=False);
    sequence_processor.get_persp_img(traj_i=traj_i, if_crop=True, resize_to=(newdx, newdy), show=False);
    #sequence_processor.perspect_plot_matplotlib(traj_i=traj_i)
    sequence_processor.perspect_plot_matplotlib(traj_i=traj_i, max_dist=50., camera = camp)